# Labolatorium nr 3

## Import bibliotek oraz ich konfiguracja

In [109]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# Matplotlib settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

## Testowane funkcje

In [140]:
def fn1(x, prec):
    return prec(np.cos(x)*np.cosh(x)-1)

def fn1_prim(x, prec):
    return prec(-np.sin(x)*np.cosh(x) + np.cos(x)*np.sinh(x))

def fn2(x, prec):
    return prec(1/x - np.tan(x))

def fn2_prim(x, prec):
    return prec(-1/(x**2) - 1/(np.cos(x)**2))

def fn3(x, prec):
    return prec(np.power(2, -x) + np.exp(x) + 2*np.cos(x) - 6)

def fn3_prim(x, prec):
    return prec(-np.log(2)*np.power(2, -x) + np.exp(x) - 2*np.sin(x))

## Zadanie 1 - Metoda bisekcji

In [146]:
def bisection_method(fn, prec, a, b, epsilon, n_max=-1):
    if n_max != -1:
        n = 1
        a = prec(a)
        b = prec(b)
        while n <= n_max:
            c = prec((b-a)/2 + a)
            if np.sign(fn(c, prec)) == np.sign(fn(a, prec)):
                a = c
            else:
                b = c
            n += 1

        return c, n

    n_max = int(np.ceil((np.log(b-a)-np.log(epsilon))/np.log(2)))
    n = 1
    a = prec(a)
    b = prec(b)
    while n <= n_max:
        c = prec((b-a)/2 + a)
        if fn(c, prec) == 0 or (b-a)/2 < epsilon:
            break
        if np.sign(fn(c, prec)) == np.sign(fn(a, prec)):
            a = c
        else:
            b = c

        n += 1

    return c, n_max


### Przykładowe wywołania

In [147]:
print(bisection_method(fn1, np.float128, np.pi*3/2, np.pi*2, 1e-7))
print(bisection_method(fn1, np.float128, np.pi*3/2, np.pi*2, 1e-15))
print(bisection_method(fn1, np.float128, np.pi*3/2, np.pi*2, 1e-33))

(4.7300407137759521003, 24)
(4.73004074486270388, 51)
(4.7300407448627040264, 111)


In [148]:
print(bisection_method(fn2, np.float128, 0.0001, np.pi/2, 1e-7))
print(bisection_method(fn2, np.float128, 0.0001, np.pi/2, 1e-15))
print(bisection_method(fn2, np.float128, 0.0001, np.pi/2, 1e-33))

(0.8603336598095483945, 24)
(0.8603335890193800182, 51)
(0.8603335890193797625, 111)


In [149]:
print(bisection_method(fn3, np.float128, 1, 3, 1e-7))
print(bisection_method(fn3, np.float128, 1, 3, 1e-15))
print(bisection_method(fn3, np.float128, 1, 3,  1e-33))

(1.829383552074432373, 25)
(1.8293836019338494125, 51)
(1.8293836019338488182, 111)


## Zadanie 2 - Metoda Newtona

In [150]:
def newton_method(fn, fn_prim, prec, a, b, n_max=-1, epsilon=0):
    x0 = bisection_method(fn, prec, a, b, 0, 1)[0]
    if n_max == -1:
        # Use epsilon
        n_max = 100
        n = 1
        x0 = prec(x0)
        while True:
            x1 = prec(x0 - fn(x0, prec)/fn_prim(x0, prec))
            if abs(x1-x0) < epsilon:
                break
            x0 = x1
            n += 1

        return x1, n

    else:
        # Use n_max
        x0 = prec(x0)
        for i in range(n_max):
            x1 = prec(x0 - fn(x0, prec)/fn_prim(x0, prec))
            x0 = x1

        return x1, n_max


### Przykładowe wywołania

In [151]:
print(newton_method(fn1, fn1_prim, np.float128, np.pi*3/2, np.pi*2, -1, 1e-15))
print(newton_method(fn1, fn1_prim, np.float128, np.pi*3/2, np.pi*2, 2))

(4.730040744862704026, 7)
(4.7837595197106524856, 2)


In [152]:
print(newton_method(fn2, fn2_prim,  np.float128, 0.0001, np.pi/2, -1, 1e-15))
print(newton_method(fn2, fn2_prim,  np.float128, 0.0001, np.pi/2, 2))

(0.8603335890193797625, 5)
(0.86033367482400727763, 2)


In [153]:
print(newton_method(fn3, fn3_prim,  np.float128, 1, 3, -1, 1e-15))
print(newton_method(fn3, fn3_prim,  np.float128, 1, 3,  2))

(1.8293836019338488182, 6)
(1.8297512018954993007, 2)


## Zadanie 3 - Metoda siecznych

In [154]:
def secant_method(fn, prec, a, b, n_max=-1, epsilon=0):
    x0 = bisection_method(fn, prec, a, b, 0, 1)[0]
    x1 = bisection_method(fn, prec, a, b, 0, 2)[0]

    if n_max == -1:
        # Use epsilon
        x0 = prec(x0)
        x1 = prec(x1)
        n = 1
        while True:
            x2 = prec(x1 - fn(x1, prec)*(x1-x0)/(fn(x1, prec)-fn(x0, prec)))
            if abs(x2-x1) < epsilon:
                break
            x0 = x1
            x1 = x2
            n += 1
        return x2, n

    else:
        # Use n_max
        x0 = prec(x0)
        x1 = prec(x1)
        for i in range(n_max):
            x2 = prec(x1 - fn(x1, prec)*(x1-x0)/(fn(x1, prec)-fn(x0, prec)))
            x0 = x1
            x1 = x2
        return x2, n_max


### Przykładowe wywołania

In [155]:
print(secant_method(fn1, np.float128, np.pi*3/2, np.pi*2, -1, 1e-15))
print(secant_method(fn1, np.float128, np.pi*3/2, np.pi*2, 2))

(4.730040744862704026, 8)
(4.7722946609192789, 2)


In [156]:
print(secant_method(fn2,  np.float128, 0.0001, np.pi/2, -1, 1e-15))
print(secant_method(fn2,  np.float128, 0.0001, np.pi/2, 2))

(0.8603335890193797625, 7)
(0.85631295909094234345, 2)


In [157]:
print(secant_method(fn3,  np.float128, 1, 3, -1, 1e-15))
print(secant_method(fn3,  np.float128, 1, 3,  2))

(1.8293836019338488181, 8)
(1.8457791814990760076, 2)
